Задание по разметке NER выполнено в рамках интенсива Трансформеры в NLP и приложениях от AI Education. В соревновании https://www.kaggle.com/competitions/litbank-ner-2024/overview требовалось побить бейзлайн организаторов без использования моделей-трансформеров.
В качестве решения была реализована модель двунаправленной LSTM на эмбеддингах GloVe.  

In [ ]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from collections import Counter
import fasttext

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.utils import class_weight

#Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler, SequentialSampler)

#BERT
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AutoModel
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers.modeling_utils import PreTrainedModel

import re
import numpy as np
import math
from functools import reduce

import time
import datetime
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.preprocessing import normalize

from gensim.models import FastText

import time
import random
import gc

In [ ]:
df = pd.read_csv('/kaggle/input/litbank-ner-2024/train_data_sent.csv',  delimiter='\t')
df.tail(20)

,ID,token,tag
179043,179043,exceptional,O
179044,179044,nature,O
179045,179045,of,O
179046,179046,his,O
179047,179047,occupation,O
179048,179048,",",O
179049,179049,and,O
179050,179050,his,O
179051,179051,advent,O
179052,179052,from,O


In [ ]:
train = df.iloc[:161138, :]
val = df.iloc[161138:, :] #около 10%, но так, чтобы не бить предложение


In [ ]:
#Сгруппируем токены обратно в предложения по точке
train_sents = []
tag_sents = []
sent = []
tag_sent = []

for tok, tag in zip(train.token.values, train.tag.values):

    if tok != '.':
        sent.append(tok)
        tag_sent.append(tag)

    else:
        sent.append(tok)
        train_sents.append(sent)
        tag_sent.append(tag)
        sent = []
        tag_sents.append(tag_sent)
        tag_sent = []




In [ ]:
val_sents = []
val_tag_sents = []
sent = []
tag_sent = []

for tok, tag in zip(val.token.values, val.tag.values):

    if tok != '.':
        sent.append(tok)
        tag_sent.append(tag)

    else:
        sent.append(tok)
        val_sents.append(sent)
        sent = []

        tag_sent.append(tag)
        val_tag_sents.append(tag_sent)
        tag_sent = []

In [ ]:
test = pd.read_csv('/kaggle/input/litbank-ner-2024/test_data_no_labels_sent.csv', delimiter = '\t')
test_sents = []
test_id_sents = []
sent = []
tag_sent = []

for ids, tok in zip(test.ID.values, test.token.values):

    if tok != '.':
        sent.append(tok)
        tag_sent.append(ids)

    else:
        sent.append(tok)
        test_sents.append(sent)
        tag_sent.append(ids)
        sent = []
        test_id_sents.append(tag_sent)
        tag_sent = []

In [ ]:
test.tail()

,ID,token
31464,31464,shocking
31465,31465,walk
31466,31466,.
31467,31467,”
31468,31468,“


In [ ]:
PAD_TAG = "PAD_TAG"
tagset = set(df.tag.values)
tagset.add(PAD_TAG)
tagset = list(tagset)

In [ ]:
ix2tag = dict(enumerate(tagset))
tag2ix = {t:ix for ix, t in ix2tag.items()}

In [ ]:
ix2tag

{0: 'I-PER',
 1: 'B-PER',
 2: 'B-ORG',
 3: 'B-LOC',
 4: 'O',
 5: 'I-LOC',
 6: 'PAD_TAG',
 7: 'I-ORG'}

In [ ]:
ix2tag[4]

'O'

In [ ]:
tag2ix

{'I-PER': 0,
 'B-PER': 1,
 'B-ORG': 2,
 'B-LOC': 3,
 'O': 4,
 'I-LOC': 5,
 'PAD_TAG': 6,
 'I-ORG': 7}

In [ ]:
print('train', len(train_sents), len(tag_sents))
print('val', len(val_sents), len(val_tag_sents))



train 5834 5834
val 598 598


In [ ]:
#максимальная длина предложения (если делать паддинг)
max = len(train_sents[0])
for sent in train_sents:

    if len(sent) > max:
        max = len(sent)
print('max', max)

max 331


In [ ]:
tag2ix

{'I-PER': 0,
 'B-PER': 1,
 'B-ORG': 2,
 'B-LOC': 3,
 'O': 4,
 'I-LOC': 5,
 'PAD_TAG': 6,
 'I-ORG': 7}

In [ ]:
train_sent_tags_digitized = [[tag2ix[tag] for tag in sent]
                             for sent in tag_sents]
val_sent_tags_digitized = [[tag2ix[tag] for tag in sent]
                             for sent in val_tag_sents]

In [ ]:
vocab = set(token for sentence_tokens in train_sents+val_sents+test_sents for token in sentence_tokens)
len(vocab)

17868

In [ ]:
vocab = list(vocab)
ix2word = dict(enumerate(vocab))
word2ix = {w:ix for ix, w in ix2word.items()}
train_sents_digitized = [[word2ix[tok] for tok in sent_toks]
                         for sent_toks in train_sents]
val_sents_digitized = [[word2ix[tok] for tok in sent_toks]
                         for sent_toks in val_sents]

In [ ]:
train_data = list(zip(train_sents_digitized, train_sent_tags_digitized))
val_data = list(zip(val_sents_digitized, val_sent_tags_digitized))
print(train_data[0])

([16368, 14769, 9349, 6414, 12802, 12689, 6260, 6239, 17074, 7039, 12574, 1829, 14923, 3422, 3700, 10576, 1709, 9405, 10875, 17074, 8227, 6239, 15887, 1709, 6980, 2631, 15706], [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 5, 4])


In [ ]:
# download glove and unzip it in Notebook.
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip glove*.zip



--2024-02-14 07:40:13--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-02-14 07:40:13--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-02-14 07:40:14--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: 'glove.6B.zip'

glov

In [ ]:
# vocab: 'the': 1, mapping of words with
# integers in seq. 1,2,3..
# embedding: 1->dense vector
def embedding_for_vocab(filepath, word_index,
                        embedding_dim):
    vocab_size = len(word_index) + 1

    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size,
                                       embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix_vocab

# matrix for vocab: word_index
embedding_dim = 300
embedding_matrix_vocab = embedding_for_vocab(
    '/kaggle/working/glove.6B.300d.txt', word2ix,
  embedding_dim)

print("Dense vector for first word is => ",
      embedding_matrix_vocab[1])

Dense vector for first word is =>  [ 1.21100001e-01 -8.81329998e-02 -1.29570007e-01  6.32470012e-01
  4.22290005e-02  1.88890006e-02 -2.96950012e-01 -2.34740004e-01
  3.92309994e-01  3.65759999e-01  1.15539998e-01 -2.83270001e-01
 -2.86460012e-01 -1.21930003e-01  2.19909996e-01 -6.96320012e-02
  3.94970000e-01 -5.80020010e-01 -4.55000013e-01  5.80730021e-01
  9.03869987e-01  4.42380011e-01  1.17540002e-01  8.07129979e-01
  2.75000006e-01 -5.70820011e-02  5.25080003e-02 -2.64739990e-01
  1.04089998e-01 -1.68349996e-01 -3.11129987e-01 -3.14549990e-02
  1.65529996e-01 -1.25320002e-01 -7.46460021e-01  8.45130009e-04
 -5.38619995e-01  5.00710011e-01 -3.87690008e-01  1.99469998e-01
  5.70029974e-01 -4.54909988e-02  3.04210000e-02 -2.05880001e-01
  1.30549997e-01 -1.96319997e-01  5.77049971e-01  1.87560007e-01
  7.22289979e-02 -1.68939993e-01  3.53590012e-01 -1.14600003e-01
  2.27599993e-01  1.27220005e-01 -4.04630005e-01  4.21059996e-01
  9.15279984e-01 -2.09279999e-01  3.36070001e-01  4.208

In [ ]:
pretrained_embeddings = torch.Tensor(embedding_matrix_vocab)

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, pretrained_embeddings):
        super(LSTMTagger, self).__init__()

        self.hidden_dim      = hidden_dim
        self.word_embeddings = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True)
        self.lstm            = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, num_layers=2, dropout = 0.3)
        self.hidden2tag      = nn.Linear(2*hidden_dim, tagset_size)

    def forward(self, words):
        embeds      = self.word_embeddings(words)
        lstm_out, _ = self.lstm(embeds.view(len(words), 1, -1))
        tag_space   = self.hidden2tag(lstm_out.view(len(words), -1))
        tag_scores  = F.log_softmax(tag_space, dim=1)

        return tag_scores

    def predict_tags(self, words):
        with torch.no_grad():
            tags_pred = model(words).numpy()
            tags_pred = np.argmax(tags_pred, axis=1)

        return tags_pred

In [ ]:
classes = np.array([i for i in tag2ix.values()])
classes


array([0, 1, 2, 3, 4, 5, 6, 7])

In [ ]:
num_pad = tag2ix['PAD_TAG']
classes = np.delete(classes, num_pad)
classes

array([0, 1, 2, 3, 4, 5, 7])

In [ ]:
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes = classes, y= np.array([item for sublist in train_sent_tags_digitized for item in sublist]))
class_weights = np.insert(class_weights, num_pad, 0)
class_weights = torch.Tensor(class_weights)
#print([round(i, 3) for i in class_weights])

In [ ]:
class_weights

tensor([2.6569e+00, 3.7675e+00, 2.7404e+02, 1.5460e+01, 1.6128e-01, 1.2238e+01,
        0.0000e+00, 1.2789e+02])

In [ ]:
model = LSTMTagger(300, 200, len(vocab), len(tagset), pretrained_embeddings)
loss_function = nn.NLLLoss(class_weights)
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
def fit(model, train_loader):

    losses = []

    predicted_labels = []
    predicted_labels_list = []
    tags_list = []

    model.train()

    loss_fn = loss_function.to(device)

    for text, tags  in train_loader:

        optimizer.zero_grad()
        tags = torch.LongTensor(tags).to(device)
        text = torch.LongTensor(text).to(device)

        outputs = model(text)
        loss = loss_fn(outputs, tags)
        loss.backward()
        losses.append(loss.item())

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        with torch.no_grad():
            predicted_labels = torch.argmax(outputs, dim=1)
            predicted_labels_list.extend(predicted_labels.tolist())
            tags_list.extend(tags.tolist())


    train_loss = np.mean(losses)
    print(classification_report(tags_list, predicted_labels_list, zero_division=0))


    return train_loss

def eval(model, valid_loader):
    model.eval()
    losses = []
    predicted_labels = []
    predicted_labels_list = []
    tags_list = []

    correct_predictions = 0

    loss_fn = loss_function = nn.NLLLoss().to(device)

    with torch.no_grad():
        for text, tags in valid_loader:
            tags = torch.LongTensor(tags).to(device)
            text = torch.LongTensor(text).to(device)

            outputs = model(text).to(device)
            loss = loss_fn(outputs, tags)


            correct_predictions += torch.sum(torch.argmax(outputs, dim=1) == tags)
            losses.append(loss.item())


            predicted_labels = torch.argmax(outputs, dim=1)
            predicted_labels_list.extend(predicted_labels.tolist())
            tags_list.extend(tags.tolist())

        print(classification_report(tags_list, predicted_labels_list, zero_division=0))

        val_loss = np.mean(losses)
        return  val_loss



def train(model, train_loader, valid_loader, optimizer, epochs):


        for epoch in range(epochs):
            print(f'Epoch {epoch + 1}/{epochs}')
            train_loss = fit(model, train_loader)
            print(f'Train loss {train_loss} ')

            val_loss = eval(model, valid_loader)
            print(f'Val loss {val_loss}')
            print('-' * 10)

            torch.save(model,f'/kaggle/working/model_epoch{epoch}.pt')
            torch.save(model.state_dict(),f'/kaggle/working/model_state_epoch{epoch}.pt')


In [ ]:
batch_tok2tag2pred = []
    batch_joint_accuracies = []
    batch_mean_accuracies = []

    for some_txt, some_tgt, tags_predicted_digitized in zip(text,
                                                            tags,
                                                            outputs.argmax(dim=-1).tolist()):
        # восстанавливаем токены, предсказанные и ожидаемые метки токенов, из их численных записей
        tokens = list(map(ix2word.get, some_text))
        tags_true = list(map(ix2tag.get, some_tgt))
        tags_pred = list(map(ix2tag.get, tags_predicted_digitized))

        tok2tag2pred = list(zip(tokens, tags_true, tags_pred))

        batch_tok2tag2pred.append(tok2tag2pred)

        correct_preds = []  # чтобы хранить верные предсказания модели в человекочитаемом виде
        actual_preds = []  # чтобы хранить *все* предсказания модели в человекочитаемом виде
        for tok, gold_tag, pred_tag in tok2tag2pred:
            batch_entry_w_prediction = (tok, gold_tag, pred_tag)
            if gold_tag == pred_tag:
                correct_preds.append(batch_entry_w_prediction)
            actual_preds.append(batch_entry_w_prediction)

        correct_preds_num = len(correct_preds)
        joint_accuracy = int(correct_preds_num == len(actual_preds))  # 1 если не было ошибок, иначе 0
        mean_accuracy = correct_preds_num/len(actual_preds) if actual_preds else 0  # доля верных ответов
        batch_joint_accuracies.append(joint_accuracy)
        batch_mean_accuracies.append(mean_accuracy)
        print(classification_report(tags_predicted_digitized, tags))

In [ ]:
device = 'cpu'
train(model, train_data, val_data, optimizer, epochs=15 )

Epoch 1/15
              precision    recall  f1-score   support

           0       0.39      0.35      0.37      8664
           1       0.24      0.60      0.34      6110
           2       0.00      0.00      0.00        84
           3       0.12      0.28      0.17      1489
           4       0.94      0.87      0.91    142730
           5       0.16      0.23      0.19      1881
           7       0.00      0.00      0.00       180

    accuracy                           0.82    161138
   macro avg       0.26      0.33      0.28    161138
weighted avg       0.87      0.82      0.84    161138

Train loss 0.9520357435593592 
              precision    recall  f1-score   support

           0       0.65      0.31      0.42      1260
           1       0.36      0.74      0.48       765
           2       0.00      0.00      0.00        14
           3       0.22      0.49      0.31       124
           4       0.94      0.93      0.93     15591
           5       0.51      0.28   

In [ ]:
train(model, train_data, val_data, optimizer, epochs=1 )

Epoch 1/1
              precision    recall  f1-score   support

           0       0.67      0.82      0.74      8664
           1       0.69      0.90      0.78      6110
           2       0.43      0.60      0.50        84
           3       0.56      0.86      0.68      1489
           4       0.99      0.94      0.96    142730
           5       0.49      0.81      0.61      1881
           7       0.43      0.56      0.49       180

    accuracy                           0.93    161138
   macro avg       0.61      0.78      0.68    161138
weighted avg       0.95      0.93      0.94    161138

Train loss 0.2455434432908563 
              precision    recall  f1-score   support

           0       0.57      0.68      0.62      1260
           1       0.59      0.74      0.66       765
           2       0.00      0.00      0.00        14
           3       0.35      0.71      0.47       124
           4       0.97      0.93      0.95     15591
           5       0.38      0.61    

In [ ]:
train(model, train_data, val_data, optimizer, epochs=3 )

Epoch 1/3
              precision    recall  f1-score   support

           0       0.55      0.74      0.63      8664
           1       0.58      0.85      0.69      6110
           2       0.31      0.35      0.33        84
           3       0.46      0.76      0.57      1489
           4       0.98      0.92      0.95    142730
           5       0.40      0.69      0.51      1881
           7       0.35      0.19      0.25       180

    accuracy                           0.90    161138
   macro avg       0.52      0.64      0.56    161138
weighted avg       0.93      0.90      0.91    161138

Train loss 0.3807654578164689 
              precision    recall  f1-score   support

           0       0.55      0.64      0.59      1260
           1       0.60      0.79      0.68       765
           2       0.00      0.00      0.00        14
           3       0.49      0.69      0.57       124
           4       0.96      0.93      0.95     15591
           5       0.44      0.55    

In [ ]:
loaded = torch.load('/kaggle/working/model_epoch14.pt')

In [ ]:
test_sents_digitized = [[word2ix[tok] for tok in sent_toks]
                         for sent_toks in test_sents]

In [ ]:
test_data = list(zip( test_id_sents, test_sents_digitized))

In [ ]:
predicted_labels=[]
predicted_labels_list=[]
ids_list=[]

for ix, text in test_data:
    text = torch.LongTensor(text)

    outputs = model(text).to(device)
    predicted_labels = torch.argmax(outputs, dim=1)
    predicted_labels_list.extend(predicted_labels.tolist())
    ids_list.extend(ix)



In [ ]:
ids_list.extend([31467, 31468])
predicted_labels_list.extend([4, 4])


In [ ]:
res = pd.DataFrame()
res['ID'] = ids_list

In [ ]:
res['pred'] =  predicted_labels_list

In [ ]:
res['tag'] = res['pred'].apply(lambda x: ix2tag[x])

In [ ]:
res = res.drop('pred', axis=1)

In [ ]:
res

,ID,tag
0,0,O
1,1,O
2,2,O
3,3,O
4,4,O
...,...,...
31464,31464,O
31465,31465,O
31466,31466,O
31467,31467,O


In [ ]:
res.to_csv('/kaggle/working/res.csv', index=False)

In [ ]:
res = res.drop('ID', axis=1)